#   Traffic Sign Recognition Project

In [1]:
import numpy as np
from skimage import color, exposure, transform

NUM_CLASSES = 43
IMG_SIZE = 48


In [2]:
def preprocess_img(img):
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[centre[0] - min_side // 2:centre[0] + min_side // 2,
              centre[1] - min_side // 2:centre[1] + min_side // 2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img, -1)

    return img

In [7]:
from skimage import io
import os
import glob

In [8]:
def get_class(img_path):
    return int(img_path.split('/')[-2])

root_dir = 'GTSRB/Final_Training/Images/'
imgs = []
labels = []

all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path))
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)

X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]

/Users/amy/anaconda3/envs/TensorFlow/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


# Model

In [5]:
import tensorflow as tf

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_data_format('channels_first')

In [13]:
def cnn_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(3, IMG_SIZE, IMG_SIZE),
                     activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

## Configuring the learning algorithm 
 
 * **Loss** function we want to optimize. We cannot use error percentage as it is not continuous and thus non          differentiable. We therefore use a proxy for it: categorical_crossentropy 
 
* **Optimizer** : We use standard stochastic gradient descent with Nesterov momentum 

* **Metric** : Since we are dealing with a classification problem, our metric is accuracy
 
 

In [15]:
from keras.optimizers import SGD
model = cnn_model()

** Let's train the model using SGD + momentum **

In [19]:
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

## Training

The model will iterate over batches of training set each size of batch_size. Gradients will be computed and updates will be made to the weigths

**Epoch**: one iteration over all the trainong set 

**Learning rate scheduler**: Decaying learning rate over the epochs

**Model checkpoint**: save the model with best validation accuracy. The network might start overfitting after certain number of epochs

Training is run until the loss converges to a constant

In [22]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [ ]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 30

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[LearningRateScheduler(lr_schedule),
                     ModelCheckpoint('model.h5', save_best_only=True)]
          )

Train on 31367 samples, validate on 7842 samples
Epoch 1/30
31367/31367 [==============================] - 615s 20ms/step - loss: 1.6899 - acc: 0.5185 - val_loss: 0.2242 - val_acc: 0.9364
Epoch 2/30
31367/31367 [==============================] - 631s 20ms/step - loss: 0.2773 - acc: 0.9149 - val_loss: 0.0824 - val_acc: 0.9759
Epoch 3/30
31367/31367 [==============================] - 646s 21ms/step - loss: 0.1594 - acc: 0.9522 - val_loss: 0.0600 - val_acc: 0.9829
Epoch 4/30
31367/31367 [==============================] - 633s 20ms/step - loss: 0.1194 - acc: 0.9629 - val_loss: 0.0401 - val_acc: 0.9884
Epoch 5/30
31367/31367 [==============================] - 621s 20ms/step - loss: 0.0954 - acc: 0.9709 - val_loss: 0.0485 - val_acc: 0.9847
Epoch 6/30
31367/31367 [==============================] - 627s 20ms/step - loss: 0.0744 - acc: 0.9773 - val_loss: 0.0281 - val_acc: 0.9927
Epoch 7/30
31367/31367 [==============================] - 619s 20ms/step - loss: 0.0611 - acc: 0.9812 - val_loss: 0.0